# 🧪 QEPC Sandbox Notebook

This notebook is a **safe playground** for experimenting with QEPC:

1. ✅ Set up the environment
2. ✅ Run diagnostics (optional, but useful)
3. ✅ Load NBA schedule (`Games.csv`)
4. ✅ Build advanced team strengths from raw stats
5. ✅ (Optional) Apply injury overrides (e.g., Haliburton out)
6. ✅ Compute game-level λ (expected points)
7. ✅ Run Poisson simulations for those games
8. ✅ Inspect win probabilities and projections

> **Tip:** Run cells **top to bottom** after restarting the kernel.


In [ ]:
# --- QEPC Sandbox: Environment Setup ---

import sys
from pathlib import Path

# 🔧 Point this at your QEPC project folder if needed
project_root = Path(r"C:\Users\wdors\qepc_project")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("Project root set to:", project_root)

# If you use a universal header like notebook_context, run it here:
try:
    from notebook_context import *
    print("✅ notebook_context imported successfully.")
except ImportError:
    print("ℹ️ notebook_context not found; continuing without it.")

# Autoload helper (in case you want it directly)
try:
    import qepc_autoload as qa
    print("✅ qepc_autoload imported as qa.")
except ImportError as e:
    print("❌ Error importing qepc_autoload:", e)


In [ ]:
# --- QEPC Sandbox: Diagnostics (Optional) ---

from qepc.utils.diagnostics import run_system_check

diagnostic_report = run_system_check()
diagnostic_report  # Just to show the dict at the end


In [ ]:
# --- QEPC Sandbox: Load NBA Schedule ---

import qepc_autoload as qa

# Load the full schedule from data/Games.csv
schedule = qa.load_nba_schedule()
print("Number of games in schedule:", len(schedule))
schedule.head()


In [ ]:
# --- QEPC Sandbox: Select Games to Model ---

# Option A: first 4 games in the schedule (example)
games_to_model = schedule.head(4).copy()

print("Using these games:")
games_to_model


In [ ]:
# --- QEPC Sandbox: Build Advanced Team Strengths ---

from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

advanced_strengths = calculate_advanced_strengths()
print("Raw advanced_strengths shape:", advanced_strengths.shape)

# Collapse to one row per team (average numeric columns)
advanced_team_strengths = (
    advanced_strengths
    .groupby("Team", as_index=False)
    .mean(numeric_only=True)
)

print("Unique teams in advanced strengths:", len(advanced_team_strengths))
advanced_team_strengths.head()


In [ ]:
# --- QEPC Sandbox: Injury Overrides (Optional) ---

import pandas as pd

inj_path = project_root / "data" / "Injury_Overrides.csv"

if inj_path.exists():
    injuries = pd.read_csv(inj_path)
    print("Loaded injury overrides:")
    display(injuries)

    # Collapse to team-level impact (multiply impacts if multiple rows per team)
    team_injury_impact = (
        injuries
        .groupby("Team", as_index=False)["Impact"]
        .prod()
        .rename(columns={"Impact": "InjuryImpact"})
    )

    print("Team-level injury impact:")
    display(team_injury_impact)

    # Merge into strengths
    inj_adjusted = advanced_team_strengths.merge(
        team_injury_impact,
        on="Team",
        how="left"
    )

    # Teams without overrides get impact = 1.0 (no change)
    inj_adjusted["InjuryImpact"] = inj_adjusted["InjuryImpact"].fillna(1.0)

    # Apply to ORtg (you can also apply to Pace if you like)
    inj_adjusted["ORtg_inj"] = inj_adjusted["ORtg"] * inj_adjusted["InjuryImpact"]

    print("Injury-adjusted team strengths (first few):")
    display(inj_adjusted.head())

    # This is the version we will feed into the lambda engine
    team_strengths_for_lambda = inj_adjusted.copy()
    team_strengths_for_lambda["ORtg"] = team_strengths_for_lambda["ORtg_inj"]

    # Clean up helper columns (optional)
    for col in ["ORtg_inj", "InjuryImpact"]:
        if col in team_strengths_for_lambda.columns:
            team_strengths_for_lambda.drop(columns=[col], inplace=True)

else:
    print("No Injury_Overrides.csv found at", inj_path)
    print("Proceeding with unadjusted advanced strengths.")
    team_strengths_for_lambda = advanced_team_strengths.copy()


In [ ]:
# --- QEPC Sandbox: Compute Lambda (Expected Points) ---

from qepc.core.lambda_engine import compute_lambda

lambda_df = compute_lambda(games_to_model, team_strengths_for_lambda)

print("Lambda dataframe columns:")
print(lambda_df.columns.tolist())

display(
    lambda_df[
        ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
    ]
)


In [ ]:
# --- QEPC Sandbox: Run QEPC Simulation ---

from qepc.core.simulator import run_qepc_simulation

sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

print("Simulation result columns:")
print(sim_results.columns.tolist())

sim_results.head()


In [ ]:
# --- QEPC Sandbox: Summary View ---

# Adjust these column names if your version uses slightly different ones
prob_cols = [c for c in sim_results.columns if "Win_Prob" in c]

summary_cols = ["Away Team", "Home Team"] + prob_cols

summary = sim_results[summary_cols].copy()

# Sort by home win probability descending if that column exists
home_prob_col = next((c for c in prob_cols if "Home" in c), None)
if home_prob_col:
    summary = summary.sort_values(home_prob_col, ascending=False)

summary.head(10)
